<a href="https://colab.research.google.com/github/sebastianlarrottadev/libro-amor/blob/main/Model_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
from transformers import BartTokenizer, BartForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments
import pandas as pd
import torch
import evaluate

# Cargar los CSVs como DataFrames
print("Cargando archivos CSV...")
train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("validation.csv")
test_df = pd.read_csv("test.csv")

# Eliminar valores nulos
train_df = train_df.dropna(subset=["article", "highlights"])
val_df = val_df.dropna(subset=["article", "highlights"])
test_df = test_df.dropna(subset=["article", "highlights"])

# Convertir a datasets de Hugging Face
print("Convirtiendo a Hugging Face Dataset...")
train_dataset = Dataset.from_pandas(train_df[["article", "highlights"]])
val_dataset = Dataset.from_pandas(val_df[["article", "highlights"]])
test_dataset = Dataset.from_pandas(test_df[["article", "highlights"]])

dataset = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

# Cargar tokenizer y modelo
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Tokenización
def tokenize(example):
    model_inputs = tokenizer(example["article"], max_length=1024, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example["highlights"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

print("Tokenizando dataset...")
tokenized_dataset = dataset.map(tokenize, batched=True)

# Cargar métrica ROUGE
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    return {k: round(v, 4) for k, v in result.items()}

# Argumentos de entrenamiento compatibles con versiones antiguas
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    logging_dir="./logs",
    save_total_limit=2,
    predict_with_generate=True,
)

# Inicializar trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Entrenar
trainer.train()

# Evaluar
trainer.evaluate()


Cargando archivos CSV...
Convirtiendo a Hugging Face Dataset...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Tokenizando dataset...


Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


...

<ipython-input-1-ee1491015a4a>:69: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: juan-sebas-23lc (juan-sebas-23lc-militar-nueva-granada) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## 🧠 Explicación del Problema

El crecimiento exponencial de contenido informativo en medios digitales ha generado una sobrecarga cognitiva en los lectores. Este proyecto busca desarrollar un sistema automático de **generación de resúmenes de noticias** que reduzca la longitud de los artículos sin perder su esencia, facilitando el consumo rápido de información.

Este tipo de sistema se enmarca en tareas de **procesamiento de lenguaje natural (PLN)**, específicamente en la **generación automática de texto (text summarization)**, utilizando modelos de lenguaje entrenados previamente (pretrained) y adaptados a nuestra tarea.

---
## 🗂️ Descripción del Dataset

Se utilizaron tres archivos CSV: `train.csv`, `val.csv` y `test.csv`. Cada uno contiene ejemplos de noticias reales con las siguientes columnas:

- `text`: Texto completo de la noticia.
- `summary`: Resumen manual realizado por editores.

| Tamaño del dataset | Cantidad de ejemplos |
|--------------------|----------------------|
| Entrenamiento      | 287,113              |
| Validación         | 13,368               |
| Prueba             | 11,490               |

Este dataset fue preprocesado y tokenizado usando Hugging Face Datasets y Tokenizers, y luego convertido a tensores para entrenamiento del modelo.

---
## 🤖 Modelo Seleccionado: BART (GPT-like para tareas de resumen)

El modelo utilizado es **BART (`facebook/bart-large-cnn`)**, un modelo encoder-decoder que combina las ventajas de BERT y GPT:

- **Encoder**: codifica el texto original.
- **Decoder**: genera el texto resumido palabra por palabra.

Aunque no es un modelo GPT puro (autoregresivo unidireccional), BART comparte sus capacidades generativas, siendo especialmente efectivo en tareas como traducción, corrección gramatical y **resumen automático**.
Modelo usado:
---
python
## from transformers import BartTokenizer, BartForConditionalGeneration

tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

---
## 📌 4. Entrenamiento

markdown
## ⚙️ Proceso de Entrenamiento

Se usó la clase `Seq2SeqTrainer` de Hugging Face, configurando los siguientes parámetros:

- Optimización: AdamW
- Learning rate: 2e-5
- Epochs: 3
- Batch size: 16
- Métrica principal: ROUGE

El entrenamiento se llevó a cabo en Google Colab con GPU, y los resultados se monitorearon usando **Weights & Biases (W&B)** para visualización en tiempo real.

---
## 📊 Resultados y Evaluación

### 🔹 Ejemplo de predicción
python
# Texto de prueba
texto_ejemplo = "El presidente anunció nuevas medidas para enfrentar la crisis económica..."

# Tokenizar y generar resumen
inputs = tokenizer(texto_ejemplo, return_tensors="pt", max_length=1024, truncation=True)
summary_ids = model.generate(inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
print(tokenizer.decode(summary_ids[0], skip_special_tokens=True))
